<a href="https://colab.research.google.com/github/Hardik-Garbyal/Spam-Detection/blob/main/D2Vmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gensim
!pip install python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 47.7 MB/s eta 0:00:00


In [ ]:
import gensim.downloader as api


In [ ]:
# wv300 = api.load('glove-twitter-100')
wv300 = api.load('word2vec-google-news-300')

[=================================================-] 99.9% 1661.7/1662.8MB downloaded


In [ ]:
import pandas as pd
import gensim
import numpy as np
import matplotlib.pyplot as plt
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.porter import PorterStemmer
from gensim.utils import simple_preprocess
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import spacy
import string

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
data = pd.read_csv("hardik.csv")

In [ ]:
data.shape

(17540, 8)

In [ ]:
data['tokenized_text'] = [simple_preprocess(line, deacc=True) for line in data['Tweet']]
print(data['tokenized_text'].head(10))

0    [it, the, everything, else, that, complicated,...
1    [eren, sent, glare, towards, mikasa, then, nod...
2    [posted, new, photo, to, facebook, http, fb, m...
3    [jan, idiot, chelsea, handler, diagnoses, trum...
4    [pedophile, anthony, weiner, is, terrified, of...
5    [ebmud, ending, penalties, for, excessive, wat...
6    [big, day, wethenorth, yyz, thesix, sunset, sk...
7    [upa, scams, to, the, tune, of, rs, lakh, cror...
8    [missing, male, tabby, cat, has, gone, missing...
9    [paraguayan, man, linked, to, hezbollah, faces...
Name: tokenized_text, dtype: object


In [ ]:
data.head()

,Id,Tweet,following,followers,actions,is_retweet,location,Type,tokenized_text
0,10091,It's the everything else that's complicated. #...,0.0,11500.0,NaN,0.0,Chicago,Quality,"[it, the, everything, else, that, complicated,..."
1,10172,Eren sent a glare towards Mikasa then nodded a...,0.0,0.0,NaN,0.0,NaN,Quality,"[eren, sent, glare, towards, mikasa, then, nod..."
2,7012,I posted a new photo to Facebook http://fb.me/...,0.0,0.0,NaN,0.0,"Scotland, U.K",Quality,"[posted, new, photo, to, facebook, http, fb, m..."
3,3697,#jan Idiot Chelsea Handler Diagnoses Trump Wit...,3319.0,611.0,294.0,0.0,"Atlanta, Ga",Spam,"[jan, idiot, chelsea, handler, diagnoses, trum..."
4,10740,Pedophile Anthony Weiner is TERRIFIED of Getti...,4840.0,1724.0,1522.0,0.0,Blumberg,Spam,"[pedophile, anthony, weiner, is, terrified, of..."


In [ ]:
porter_stemmer = PorterStemmer()
# Get the stemmed_tokens
data['st'] = [[porter_stemmer.stem(word) for word in tokens] for tokens in data['tokenized_text'] ]
# data.head(10)

In [ ]:
# replace the missing values with new values
data = data.fillna({
'Id	':0,
'Tweet': "No Tweet",
'following': 0,
'followers': 0,
'actions': 0,
'is_retweet': "Nothing",
'location': "No place",
'Type': "Not determined"
})

In [ ]:
data.tail()

,Id,Tweet,following,followers,actions,is_retweet,location,Type,tokenized_text,st
17535,6801,This is the 2nd time we have tried 2 contact u...,0.0,0.0,0.0,Nothing,No place,Spam,"[this, is, the, nd, time, we, have, tried, con...","[thi, is, the, nd, time, we, have, tri, contac..."
17536,6802,Will Ã_ b going to esplanade fr home?,0.0,0.0,0.0,Nothing,No place,Quality,"[will, going, to, esplanade, fr, home]","[will, go, to, esplanad, fr, home]"
17537,6803,"Pity, * was in mood for that. So...any other s...",0.0,0.0,0.0,Nothing,No place,Quality,"[pity, was, in, mood, for, that, so, any, othe...","[piti, wa, in, mood, for, that, so, ani, other..."
17538,6804,The guy did some bitching but I acted like i'd...,0.0,0.0,0.0,Nothing,No place,Quality,"[the, guy, did, some, bitching, but, acted, li...","[the, gui, did, some, bitch, but, act, like, b..."
17539,6805,Rofl. Its true to its name,0.0,0.0,0.0,Nothing,No place,Quality,"[rofl, its, true, to, its, name]","[rofl, it, true, to, it, name]"


In [ ]:
data.loc[data['Type'] == 'Spam', 'Type',] = 0
data.loc[data['Type'] == 'Quality', 'Type',] = 1

In [ ]:
X = data['st']
Y = data['Type']
forw2v = data['st']
ford2v = data['st']

Word2Vec

In [ ]:
def sent_vec(sent):
    vector_size = wv300.vector_size
    wv_res = np.zeros(vector_size)
    # print(wv_res)
    count = 1
    for w in sent:
        if w in wv300:
            count += 1
            wv_res += wv300[w]
    wv_res = wv_res/count
    return wv_res

In [ ]:
data['word2vec'] = data['tokenized_text'].apply(sent_vec)

In [ ]:
data.head()

,Id,Tweet,following,followers,actions,is_retweet,location,Type,tokenized_text,st,word2vec
0,10091,It's the everything else that's complicated. #...,0.0,11500.0,0.0,0.0,Chicago,1,"[it, the, everything, else, that, complicated,...","[it, the, everyth, els, that, complic, pesummi...","[0.03530544704861111, -0.020282533433702257, -..."
1,10172,Eren sent a glare towards Mikasa then nodded a...,0.0,0.0,0.0,0.0,No place,1,"[eren, sent, glare, towards, mikasa, then, nod...","[eren, sent, glare, toward, mikasa, then, nod,...","[0.047491374768708884, 0.045654296875, 0.04210..."
2,7012,I posted a new photo to Facebook http://fb.me/...,0.0,0.0,0.0,0.0,"Scotland, U.K",1,"[posted, new, photo, to, facebook, http, fb, m...","[post, new, photo, to, facebook, http, fb, me,...","[-0.03238254123263889, 0.011623806423611112, 0..."
3,3697,#jan Idiot Chelsea Handler Diagnoses Trump Wit...,3319.0,611.0,294.0,0.0,"Atlanta, Ga",0,"[jan, idiot, chelsea, handler, diagnoses, trum...","[jan, idiot, chelsea, handler, diagnos, trump,...","[-0.058236929086538464, 0.053560697115384616, ..."
4,10740,Pedophile Anthony Weiner is TERRIFIED of Getti...,4840.0,1724.0,1522.0,0.0,Blumberg,0,"[pedophile, anthony, weiner, is, terrified, of...","[pedophil, anthoni, weiner, is, terrifi, of, g...","[0.0031962076822916667, 0.030069986979166668, ..."


DOC2vec processing

In [ ]:
tagged_documents = [TaggedDocument(words=word_tokenize(doc.lower()), tags=[str(i)]) for i, doc in enumerate('tokenized_text')]

In [ ]:
data['doc2vec'] = X

In [ ]:
d2v_model = Doc2Vec(vector_size=300, min_count=1, epochs=1000)
d2v_model.build_vocab(tagged_documents)
d2v_model.train(tagged_documents, total_examples=d2v_model.corpus_count, epochs=d2v_model.epochs)

In [ ]:
for i in range(len(X)):
  X[i] = d2v_model.infer_vector(X[i])

<ipython-input-22-dec59d23e09c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[i] = d2v_model.infer_vector(X[i])


In [ ]:
data['doc2vec'] = X

In [ ]:
data.head()

,Id,Tweet,following,followers,actions,is_retweet,location,Type,tokenized_text,st,word2vec,doc2vec
0,10091,It's the everything else that's complicated. #...,0.0,11500.0,0.0,0.0,Chicago,1,"[it, the, everything, else, that, complicated,...","[-0.0015131686, -0.0003906853, -0.00024941444,...","[0.03530544704861111, -0.020282533433702257, -...","[-0.0015131686, -0.0003906853, -0.00024941444,..."
1,10172,Eren sent a glare towards Mikasa then nodded a...,0.0,0.0,0.0,0.0,No place,1,"[eren, sent, glare, towards, mikasa, then, nod...","[-0.00067427603, -0.0005292704, 2.5632182e-05,...","[0.047491374768708884, 0.045654296875, 0.04210...","[-0.00067427603, -0.0005292704, 2.5632182e-05,..."
2,7012,I posted a new photo to Facebook http://fb.me/...,0.0,0.0,0.0,0.0,"Scotland, U.K",1,"[posted, new, photo, to, facebook, http, fb, m...","[0.00036508142, 0.00016559183, -0.0009807701, ...","[-0.03238254123263889, 0.011623806423611112, 0...","[0.00036508142, 0.00016559183, -0.0009807701, ..."
3,3697,#jan Idiot Chelsea Handler Diagnoses Trump Wit...,3319.0,611.0,294.0,0.0,"Atlanta, Ga",0,"[jan, idiot, chelsea, handler, diagnoses, trum...","[-0.00095282256, 0.0012629395, -0.00081277697,...","[-0.058236929086538464, 0.053560697115384616, ...","[-0.00095282256, 0.0012629395, -0.00081277697,..."
4,10740,Pedophile Anthony Weiner is TERRIFIED of Getti...,4840.0,1724.0,1522.0,0.0,Blumberg,0,"[pedophile, anthony, weiner, is, terrified, of...","[0.00038281005, 0.0011373558, -8.6418586e-05, ...","[0.0031962076822916667, 0.030069986979166668, ...","[0.00038281005, 0.0011373558, -8.6418586e-05, ..."


In [ ]:
data.tail()

,Id,Tweet,following,followers,actions,is_retweet,location,Type,tokenized_text,st,word2vec,doc2vec
17535,6801,This is the 2nd time we have tried 2 contact u...,0.0,0.0,0.0,Nothing,No place,0,"[this, is, the, nd, time, we, have, tried, con...","[0.0013460412, -0.00013797065, -0.001036924, -...","[0.02329430213341346, 0.04472901270939754, 0.0...","[0.0013460412, -0.00013797065, -0.001036924, -..."
17536,6802,Will Ã_ b going to esplanade fr home?,0.0,0.0,0.0,Nothing,No place,1,"[will, going, to, esplanade, fr, home]","[-0.0013231902, 0.0004355087, -0.00038908163, ...","[0.05999755859375, 0.094482421875, 0.100921630...","[-0.0013231902, 0.0004355087, -0.00038908163, ..."
17537,6803,"Pity, * was in mood for that. So...any other s...",0.0,0.0,0.0,Nothing,No place,1,"[pity, was, in, mood, for, that, so, any, othe...","[0.00040432057, -0.00048527768, 0.0010187362, ...","[0.004921653053977273, 0.028592196377840908, 0...","[0.00040432057, -0.00048527768, 0.0010187362, ..."
17538,6804,The guy did some bitching but I acted like i'd...,0.0,0.0,0.0,Nothing,No place,1,"[the, guy, did, some, bitching, but, acted, li...","[0.00066766085, -0.00075185805, 0.00028772812,...","[0.07886007557744565, 0.03753985529360564, 0.0...","[0.00066766085, -0.00075185805, 0.00028772812,..."
17539,6805,Rofl. Its true to its name,0.0,0.0,0.0,Nothing,No place,1,"[rofl, its, true, to, its, name]","[-0.00089864375, 0.0011898791, -0.00019223482,...","[0.059855143229166664, 0.10623677571614583, 0....","[-0.00089864375, 0.0011898791, -0.00019223482,..."


In [ ]:
word2veclist = data['word2vec'].to_list()
spamlist = Y.to_list()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(word2veclist, spamlist, test_size=0.2)

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(word2veclist, spamlist, test_size=0.2)

# Create an SVM classifier
clf = svm.SVC(kernel='linear')

# Train the classifier
clf.fit(X_train, Y_train)

# Make predictions on the test set
Y_pred = clf.predict(X_test)

# Evaluate the classifier
accuracy = clf.score(X_test, Y_test)
print("Accuracy:", accuracy)


Accuracy: 0.9167616875712656


doc2vec

In [ ]:
doc2veclist = data['doc2vec'].to_list()
spamlist = Y.to_list()
from sklearn.model_selection import train_test_split
A_train, A_test, B_train, B_test = train_test_split(doc2veclist, spamlist, test_size=0.2)
# from sklearn.linear_model import LogisticRegression
# classifier = LogisticRegression()

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
A_train, A_test, B_train, B_test = train_test_split(doc2veclist, spamlist, test_size=0.2)
# X_train, X_test, Y_train, Y_test = train_test_split(word2veclist, spamlist, test_size=0.2)

# Create an SVM classifier
clf = svm.SVC(kernel='linear')

# Train the classifier
clf.fit(A_train, B_train)

# Make predictions on the test set
Y_pred = clf.predict(A_test)

# Evaluate the classifier
accuracy = clf.score(A_test, B_test)
print("Accuracy:", accuracy)


Accuracy: 0.6259977194982896


In [ ]:
data.head()

,Id,Tweet,following,followers,actions,is_retweet,location,Type,tokenized_text,st,word2vec,doc2vec
0,10091,It's the everything else that's complicated. #...,0.0,11500.0,0.0,0.0,Chicago,1,"[it, the, everything, else, that, complicated,...","[-0.0015131686, -0.0003906853, -0.00024941444,...","[0.03530544704861111, -0.020282533433702257, -...","[-0.0015131686, -0.0003906853, -0.00024941444,..."
1,10172,Eren sent a glare towards Mikasa then nodded a...,0.0,0.0,0.0,0.0,No place,1,"[eren, sent, glare, towards, mikasa, then, nod...","[-0.00067427603, -0.0005292704, 2.5632182e-05,...","[0.047491374768708884, 0.045654296875, 0.04210...","[-0.00067427603, -0.0005292704, 2.5632182e-05,..."
2,7012,I posted a new photo to Facebook http://fb.me/...,0.0,0.0,0.0,0.0,"Scotland, U.K",1,"[posted, new, photo, to, facebook, http, fb, m...","[0.00036508142, 0.00016559183, -0.0009807701, ...","[-0.03238254123263889, 0.011623806423611112, 0...","[0.00036508142, 0.00016559183, -0.0009807701, ..."
3,3697,#jan Idiot Chelsea Handler Diagnoses Trump Wit...,3319.0,611.0,294.0,0.0,"Atlanta, Ga",0,"[jan, idiot, chelsea, handler, diagnoses, trum...","[-0.00095282256, 0.0012629395, -0.00081277697,...","[-0.058236929086538464, 0.053560697115384616, ...","[-0.00095282256, 0.0012629395, -0.00081277697,..."
4,10740,Pedophile Anthony Weiner is TERRIFIED of Getti...,4840.0,1724.0,1522.0,0.0,Blumberg,0,"[pedophile, anthony, weiner, is, terrified, of...","[0.00038281005, 0.0011373558, -8.6418586e-05, ...","[0.0031962076822916667, 0.030069986979166668, ...","[0.00038281005, 0.0011373558, -8.6418586e-05, ..."


Merging

In [ ]:
import numpy as np

def merge_vectors(row, weight_w2v, weight_d2v):
    vector_w2v = row['word2vec']
    vector_d2v = row['doc2vec']
    merged_vector = (vector_w2v * weight_w2v) + (vector_d2v * weight_d2v)
    return merged_vector

# Assuming you have a DataFrame named 'df' with 'w2v' and 'd2v' columns

# Define the weights for w2v and d2v vectors
weight_w2v = 0.85 #change the weights and check the accuracy
weight_d2v = 0.15

# Apply the merge_vectors function to create a new column 'merged_vector'
data['weighted_merged_vector'] = data.apply(merge_vectors, args=(weight_w2v, weight_d2v), axis=1)


In [ ]:
data.head()

,Id,Tweet,following,followers,actions,is_retweet,location,Type,tokenized_text,st,word2vec,doc2vec,weighted_merged_vector
0,10091,It's the everything else that's complicated. #...,0.0,11500.0,0.0,0.0,Chicago,1,"[it, the, everything, else, that, complicated,...","[-0.0015131686, -0.0003906853, -0.00024941444,...","[0.03530544704861111, -0.020282533433702257, -...","[-0.0015131686, -0.0003906853, -0.00024941444,...","[0.029782654703659418, -0.017298756218628517, ..."
1,10172,Eren sent a glare towards Mikasa then nodded a...,0.0,0.0,0.0,0.0,No place,1,"[eren, sent, glare, towards, mikasa, then, nod...","[-0.00067427603, -0.0005292704, 2.5632182e-05,...","[0.047491374768708884, 0.045654296875, 0.04210...","[-0.00067427603, -0.0005292704, 2.5632182e-05,...","[0.04026652714735399, 0.03872676177998073, 0.0..."
2,7012,I posted a new photo to Facebook http://fb.me/...,0.0,0.0,0.0,0.0,"Scotland, U.K",1,"[posted, new, photo, to, facebook, http, fb, m...","[0.00036508142, 0.00016559183, -0.0009807701, ...","[-0.03238254123263889, 0.011623806423611112, 0...","[0.00036508142, 0.00016559183, -0.0009807701, ...","[-0.02747039783288427, 0.009905074235131098, 0..."
3,3697,#jan Idiot Chelsea Handler Diagnoses Trump Wit...,3319.0,611.0,294.0,0.0,"Atlanta, Ga",0,"[jan, idiot, chelsea, handler, diagnoses, trum...","[-0.00095282256, 0.0012629395, -0.00081277697,...","[-0.058236929086538464, 0.053560697115384616, ...","[-0.00095282256, 0.0012629395, -0.00081277697,...","[-0.04964431311975484, 0.045716033484159896, -..."
4,10740,Pedophile Anthony Weiner is TERRIFIED of Getti...,4840.0,1724.0,1522.0,0.0,Blumberg,0,"[pedophile, anthony, weiner, is, terrified, of...","[0.00038281005, 0.0011373558, -8.6418586e-05, ...","[0.0031962076822916667, 0.030069986979166668, ...","[0.00038281005, 0.0011373558, -8.6418586e-05, ...","[0.0027741980381930866, 0.025730092312248112, ..."


In [ ]:
weights = data['weighted_merged_vector'].to_list()
spamlist = Y.to_list()


In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
# Split the data into training and testing sets
A_train, A_test, B_train, B_test = train_test_split(weights, spamlist, test_size=0.2)
# X_train, X_test, Y_train, Y_test = train_test_split(word2veclist, spamlist, test_size=0.2)

# Create an SVM classifier
clf = SVC()
clf = svm.SVC(kernel='linear')

# Train the classifier
clf.fit(A_train, B_train)

# Make predictions on the test set
Y_pred = clf.predict(A_test)

# Evaluate the classifier
accuracy = clf.score(A_test, B_test)
print("Accuracy:", accuracy)


Accuracy: 0.9116305587229191
